In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, GRU, Dense, Dropout, Bidirectional, GlobalMaxPooling1D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, hamming_loss, accuracy_score, f1_score, precision_score, recall_score
import matplotlib.pyplot as plt
import time
import os

### --- 1. Load và chuẩn bị dữ liệu ---

In [ ]:
# Giả sử file data của bạn tên là 'comments.csv'
try:
    df = pd.read_csv('comments.csv')
    print("Data loaded successfully.")
except FileNotFoundError:
    print("Error: 'comments.csv' not found. Please provide the correct file path.")
    # Tạo DataFrame mẫu nếu không có file
    data = {'product_id': ['honor-x8b', 'honor-x8b', 'honor-x8b', 'iphone-12', 'iphone-12', 'iphone-12', 'iphone-12', 'iphone-12', 'iphone-12', 'iphone-12', 'iphone-12', 'iphone-12'],
            'comment': ['Đừng mua, ko có linh kiện.', 'Mua online bên này báo ngoài ip ra thì hàng ko có full box...', 'Mạng yếu hay mất sóng...', 'Mua máy lướt được 2 hôm đã lỗi loa ngoài...', 'Mới mua mà dễ nóng quá...', 'Hàng trưng bày. Sạc 4x lần...', 'Mặt kính cảm ứng cường lực nhưng ko chống sước...', 'Mai mốt mang đi bảo hành đây...', 'Mới mua được 3 tháng bây giờ phát hiện ra bức xạ...', 'Mới mua được 1 tháng thì xảy ra tình trạng tụt pin...', 'Bắt mạng rất yếu.', 'Tệ pin dại quá'],
            'comment_clean_stage1': ['đừng mua, không có linh kiện.', 'mua online bên này báo ngoài iphone ra thì hàng không có full box...', 'mạng yếu hay mất sóng...', 'mua máy lướt được hai hôm đã lỗi loa ngoài...', 'mới mua mà dễ nóng quá...', 'hàng trưng bày. sạc 4 lần...', 'mặt kính cảm ứng cường lực nhưng không chống sước...', 'mai mốt mang đi bảo hành đây...', 'mới mua được ba tháng bây giờ phát hiện ra bức xạ...', 'mới mua được một tháng thì xảy ra tình trạng tụt pin...', 'bắt mạng rất yếu.', 'tệ pin dại quá!.'],
            'comment_clean_stage2': ['đừng mua không linh_kiện', 'mua online bên báo iphone hàng không full box máy tạm ổn hơi nhẹ hơn xiaomi tuy_nhiên gọi mạng wifi 4g đứng hình suốt mà check máy vẫn gọi ổn không phải đường truyền mạng kém không thời_gian cửa_hàng giờ gọi hỏi bảo cửa_hàng bảo_hành mới mua được hai hôm giờ muốn đổi iphone 13 được đổi không hay hỗ_trợ đổi không shop máy gọi không thấy hình chập_chờn hoài dù mạng vẫn ổn_định sao', 'mạng yếu hay mất sóng đổi máy được không', 'mua máy lướt được hai hôm lỗi loa lúc bán nhân_viên không báo lỗi loa phải gửi đi bảo_hành mất thời_gian tốn tiền dán cường_lực pda', 'mới mua mà dễ nóng quá sạc nhất_là bật máy_ảnh phút nóng', 'hàng trưng_bày sạc bốn lần mà pin tụt', 'mặt kính cảm_ứng cường_lực nhưng không chống xước dùng được hai tháng màn_hình xước không chỗ trống thua màn_hình iphone', 'mai_mốt mang đi bảo_hành tự_nhiên sọc màn cho_dù không rơi', 'mới mua được tháng phát_hiện bức_xạ đổi được không', 'mới mua được tháng xảy tình_trạng tụt pin đêm tắt toàn_bộ ứng_dụng mạng bluetooth chả nổi quá tệ', 'bắt mạng rất yếu', 'tệ pin dại quá'],
            'rating': [1, 2, 1, 1, 2, 1, 1, 1, 2, 2, 2, 1],
            'positive': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
            'negative': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
    df = pd.DataFrame(data)
    print("Using sample data.")


# Đảm bảo cột comment sạch không bị NaN
df['comment_clean_stage2'] = df['comment_clean_stage2'].fillna('').astype(str)

# Tạo nhãn đa nhãn
labels = df[['positive', 'negative']].values
comments = df['comment_clean_stage2'].values

### --- 2. Tokenization và Padding ---

In [ ]:
MAX_NUM_WORDS = 10000  # Giới hạn số lượng từ trong từ điển
MAX_SEQUENCE_LENGTH = 150 # Độ dài tối đa của một chuỗi (comment)
EMBEDDING_DIM = 128    # Kích thước vector embedding

tokenizer = Tokenizer(num_words=MAX_NUM_WORDS, oov_token="<OOV>")
tokenizer.fit_on_texts(comments)
sequences = tokenizer.texts_to_sequences(comments)

word_index = tokenizer.word_index
print(f"Found {len(word_index)} unique tokens.")

data_padded = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post', truncating='post')

### --- 3. Chia dữ liệu ---

In [ ]:
X_train, X_temp, y_train, y_temp = train_test_split(data_padded, labels, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

print(f"Train data shape: {X_train.shape}, Train labels shape: {y_train.shape}")
print(f"Validation data shape: {X_val.shape}, Val labels shape: {y_val.shape}")
print(f"Test data shape: {X_test.shape}, Test labels shape: {y_test.shape}")

### --- 4. Định nghĩa các mô hình ---

In [ ]:
def build_bilstm_model(vocab_size, embedding_dim, max_length, num_labels=2):
    model = Sequential([
        Embedding(input_dim=vocab_size,
                  output_dim=embedding_dim,
                  input_length=max_length),
        Bidirectional(LSTM(64, return_sequences=True)), # Giữ lại chuỗi cho lớp tiếp theo hoặc pooling
        # GlobalMaxPooling1D(), # Có thể thêm pooling nếu muốn
        Bidirectional(LSTM(32)), # Lớp LSTM cuối không cần return_sequences=True
        Dropout(0.5),
        Dense(32, activation='relu'),
        Dropout(0.3),
        Dense(num_labels, activation='sigmoid') # Sigmoid cho multi-label
    ])
    model.compile(loss='binary_crossentropy', # Phù hợp cho multi-label
                  optimizer='adam',
                  metrics=['binary_accuracy', tf.keras.metrics.AUC(name='auc')]) # AUC là metric tốt
    return model

def build_bigru_model(vocab_size, embedding_dim, max_length, num_labels=2):
    model = Sequential([
        Embedding(input_dim=vocab_size,
                  output_dim=embedding_dim,
                  input_length=max_length),
        Bidirectional(GRU(64, return_sequences=True)),
        # GlobalMaxPooling1D(),
        Bidirectional(GRU(32)),
        Dropout(0.5),
        Dense(32, activation='relu'),
        Dropout(0.3),
        Dense(num_labels, activation='sigmoid')
    ])
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['binary_accuracy', tf.keras.metrics.AUC(name='auc')])
    return model

### --- 5. Huấn luyện và Đánh giá ---

In [ ]:
models_to_train = {
    "BiLSTM": build_bilstm_model(MAX_NUM_WORDS, EMBEDDING_DIM, MAX_SEQUENCE_LENGTH),
    "BiGRU": build_bigru_model(MAX_NUM_WORDS, EMBEDDING_DIM, MAX_SEQUENCE_LENGTH)
}

results = {}
histories = {}
training_times = {}
prediction_times = {}

BATCH_SIZE = 32 # Giảm nếu gặp lỗi OOM (Out of Memory)
EPOCHS = 50     # Số epochs lớn, dùng EarlyStopping để dừng sớm

# Tạo thư mục lưu model weights
if not os.path.exists('model_checkpoints'):
    os.makedirs('model_checkpoints')

for model_name, model in models_to_train.items():
    print(f"\n--- Training {model_name} ---")
    model.summary()

    # Callbacks
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    model_checkpoint = ModelCheckpoint(
        filepath=f'model_checkpoints/{model_name}_best.keras', # Sử dụng định dạng .keras mới
        save_best_only=True,
        monitor='val_loss',
        mode='min'
    )

    start_time = time.time()
    history = model.fit(X_train, y_train,
                        epochs=EPOCHS,
                        batch_size=BATCH_SIZE,
                        validation_data=(X_val, y_val),
                        callbacks=[early_stopping, model_checkpoint],
                        verbose=2) # verbose=2 cho ít output hơn
    end_time = time.time()
    training_times[model_name] = end_time - start_time
    histories[model_name] = history

    print(f"\n--- Evaluating {model_name} on Test Set ---")
    # Load lại best model đã lưu
    best_model = tf.keras.models.load_model(f'model_checkpoints/{model_name}_best.keras')

    loss, binary_acc, auc = best_model.evaluate(X_test, y_test, verbose=0)
    print(f"{model_name} Test Loss: {loss:.4f}")
    print(f"{model_name} Test Binary Accuracy: {binary_acc:.4f}")
    print(f"{model_name} Test AUC: {auc:.4f}")

    # Dự đoán và tính toán các metrics khác
    start_pred_time = time.time()
    y_pred_prob = best_model.predict(X_test)
    end_pred_time = time.time()
    prediction_times[model_name] = end_pred_time - start_pred_time

    y_pred = (y_pred_prob > 0.5).astype(int) # Ngưỡng 0.5 để chuyển prob -> class

    # Metrics chi tiết (Sklearn)
    accuracy = accuracy_score(y_test, y_pred) # Subset accuracy
    hamming = hamming_loss(y_test, y_pred)
    f1_micro = f1_score(y_test, y_pred, average='micro')
    f1_macro = f1_score(y_test, y_pred, average='macro')
    f1_samples = f1_score(y_test, y_pred, average='samples')
    precision_macro = precision_score(y_test, y_pred, average='macro', zero_division=0)
    recall_macro = recall_score(y_test, y_pred, average='macro', zero_division=0)

    print(f"\n{model_name} Detailed Test Metrics:")
    print(f"  Subset Accuracy (Exact Match Ratio): {accuracy:.4f}")
    print(f"  Hamming Loss (Lower is better): {hamming:.4f}")
    print(f"  F1 Score (Micro): {f1_micro:.4f}")
    print(f"  F1 Score (Macro): {f1_macro:.4f}")
    print(f"  F1 Score (Samples): {f1_samples:.4f}") # Trung bình F1 của từng mẫu
    print(f"  Precision (Macro): {precision_macro:.4f}")
    print(f"  Recall (Macro): {recall_macro:.4f}")
    print("\nClassification Report (Label-wise):")
    print(classification_report(y_test, y_pred, target_names=['positive', 'negative'], zero_division=0))

    results[model_name] = {
        'loss': loss,
        'binary_accuracy': binary_acc,
        'auc': auc,
        'subset_accuracy': accuracy,
        'hamming_loss': hamming,
        'f1_micro': f1_micro,
        'f1_macro': f1_macro,
        'f1_samples': f1_samples,
        'precision_macro': precision_macro,
        'recall_macro': recall_macro,
        'training_time': training_times[model_name],
        'prediction_time_per_batch': prediction_times[model_name] / len(X_test) if len(X_test)>0 else 0
    }

### --- 6. So sánh và Vẽ biểu đồ ---

In [ ]:
print("\n--- Comparison Summary ---")
results_df = pd.DataFrame(results).T # Transpose để model thành hàng
print(results_df)

# Vẽ biểu đồ lịch sử huấn luyện (loss và accuracy)
def plot_history(histories, metric='binary_accuracy', auc_metric='auc'):
    plt.figure(figsize=(12, 5))

    # Plot Training & Validation Accuracy
    plt.subplot(1, 2, 1)
    for model_name, history in histories.items():
        plt.plot(history.history[metric], label=f'{model_name} Train Acc')
        plt.plot(history.history[f'val_{metric}'], label=f'{model_name} Val Acc', linestyle='--')
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend()
    plt.grid(True)

    # Plot Training & Validation Loss
    plt.subplot(1, 2, 2)
    for model_name, history in histories.items():
        plt.plot(history.history['loss'], label=f'{model_name} Train Loss')
        plt.plot(history.history['val_loss'], label=f'{model_name} Val Loss', linestyle='--')
        # Optional: Plot AUC if available
        if auc_metric in history.history:
            plt.plot(history.history[auc_metric], label=f'{model_name} Train AUC', linestyle=':')
        if f'val_{auc_metric}' in history.history:
             plt.plot(history.history[f'val_{auc_metric}'], label=f'{model_name} Val AUC', linestyle='-.')

    plt.title('Model Loss & AUC')
    plt.ylabel('Loss / AUC')
    plt.xlabel('Epoch')
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    plt.show()

plot_history(histories)

### --- 7. Ví dụ dự đoán trên comment mới ---

In [ ]:
def predict_sentiment(text, model, tokenizer, max_length):
    # Làm sạch và chuẩn bị text giống như lúc train (giả sử đã có hàm clean_text)
    # clean_text = text # Tạm thời bỏ qua bước làm sạch phức tạp nếu không có sẵn
    sequence = tokenizer.texts_to_sequences([text])
    padded_sequence = pad_sequences(sequence, maxlen=max_length, padding='post', truncating='post')
    prediction = model.predict(padded_sequence)
    labels = (prediction > 0.5).astype(int)[0] # Lấy dự đoán cho mẫu đầu tiên (và duy nhất)
    return {"positive": labels[0], "negative": labels[1]}, prediction[0]

# Chọn model tốt nhất dựa trên validation (hoặc test) performance để dự đoán
best_model_name = results_df['f1_macro'].idxmax() # Ví dụ chọn theo F1 Macro cao nhất
print(f"\nUsing best model for prediction: {best_model_name}")
best_model = tf.keras.models.load_model(f'model_checkpoints/{best_model_name}_best.keras')

# Ví dụ
new_comment_positive = "điện thoại này dùng rất mượt pin trâu chụp ảnh đẹp lắm"
new_comment_negative = "máy lag quá dùng chán không tả nổi hay bị sập nguồn"
new_comment_mixed = "camera ổn nhưng pin tụt nhanh kinh khủng"
new_comment_neutral = "giao hàng đúng hẹn đóng gói cẩn thận" # Mô hình có thể khó đoán đúng neutral

for comment in [new_comment_positive, new_comment_negative, new_comment_mixed, new_comment_neutral]:
    pred_labels, pred_probs = predict_sentiment(comment, best_model, tokenizer, MAX_SEQUENCE_LENGTH)
    print(f"\nComment: '{comment}'")
    print(f"  Predicted Probabilities (Pos, Neg): ({pred_probs[0]:.3f}, {pred_probs[1]:.3f})")
    print(f"  Predicted Labels: {pred_labels}")